In [1]:
import numpy as np
import pandas as pd

In [8]:
data = pd.read_csv('/content/spam (1).csv',encoding='Windows-1252')

In [9]:
pd.set_option('display.max_colwidth', None)
data.head()

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives around here though",NaN,NaN,NaN


In [11]:
pd.set_option('display.max_colwidth', None)
messages = data[['v1','v2']]
messages.columns = ["label", "text"]
messages.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [12]:
data.shape

(5572, 5)

In [13]:
messages['label']= messages['label'].map({"ham":1, "spam":0})

<ipython-input-13-862f2c573ca0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  messages['label']= messages['label'].map({"ham":1, "spam":0})


In [15]:
messages.head()

,label,text
0,1,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
1,1,Ok lar... Joking wif u oni...
2,0,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
3,1,U dun say so early hor... U c already then say...
4,1,"Nah I don't think he goes to usf, he lives around here though"


In [17]:
messages.label.value_counts()

1    4825
0     747
Name: label, dtype: int64

## **Data Pre-processing**

In [18]:
import string
def remove_punctuation(text):
    punctuationfree="".join([i for i in text if i not in string.punctuation])
    return punctuationfree

In [19]:
import nltk
nltk.download('punkt')
def tokenization(text):
    words = nltk.word_tokenize(text)
    return words

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [20]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('english')
def remove_stopwords(text):
    output= [i for i in text if i not in stopwords]
    return output

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
nltk.download('omw-1.4')
wordnet_lemmatizer = WordNetLemmatizer()
def lemmatizer(text):
  lemm_text = [wordnet_lemmatizer.lemmatize(word) for word in text]
  return lemm_text

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [25]:
def preprocess(mesaages_col):
  corpus = []
  for item in mesaages_col:
    new_item = remove_punctuation(item)
    new_item = new_item.lower()
    new_item = tokenization(new_item)
    new_item = remove_stopwords(new_item)
    new_item = lemmatizer(new_item)
    corpus.append(' '.join(str(x) for x in new_item))
  return corpus

In [26]:
corpus = preprocess(messages.text)

In [27]:
corpus[0:10]

['go jurong point crazy available bugis n great world la e buffet cine got amore wat',
 'ok lar joking wif u oni',
 'free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s',
 'u dun say early hor u c already say',
 'nah dont think go usf life around though',
 'freemsg hey darling 3 week word back id like fun still tb ok xxx std chgs send å£150 rcv',
 'even brother like speak treat like aid patent',
 'per request melle melle oru minnaminunginte nurungu vettam set callertune caller press 9 copy friend callertune',
 'winner valued network customer selected receivea å£900 prize reward claim call 09061701461 claim code kl341 valid 12 hour',
 'mobile 11 month u r entitled update latest colour mobile camera free call mobile update co free 08002986030']

In [30]:
messages.label.head(10)

0    1
1    1
2    0
3    1
4    1
5    0
6    1
7    1
8    0
9    0
Name: label, dtype: int64

## **Bag-of-words**

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(ngram_range=(1,2))
traindata = cv.fit_transform(corpus)
X = traindata
y = messages.label

In [32]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 100)
clf.fit(X, y)

RandomForestClassifier()

In [33]:
from sklearn import metrics
y_pred = clf.predict(X) 
metrics.accuracy_score(y_pred, y)

1.0

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split (corpus, messages['label'] , test_size=0.2)

In [43]:
testdata = cv.transform(X_test)

In [44]:
predictions = clf.predict(testdata)

In [45]:
metrics.accuracy_score(y_test,predictions)

1.0

In [55]:
def find_spam_ham(input):
  input = cv.transform(preprocess(input))
  prediction = clf.predict(input)
  if prediction == 0: 
    print('spam message')
  if prediction == 1:
    print('ham message')

In [56]:
input = ["You’ve won a prize! Go to [link] to claim your $500 Amazon gift card."]
find_spam_ham(input)

spam message


In [57]:
input = ["Free entry in 2 a wkly comp to win  tkts!!"]
find_spam_ham(input)

ham message


In [58]:
input = ["I've been searching for the right words to tha..."]
find_spam_ham(input)

ham message


In [61]:
input = ["December only! Had your mobile 11mths+? You are entitled to update to the latest colour camera mobile for Free! Call The Mobile Update Co FREE on 08002986906."]
find_spam_ham(input)


spam message
